### Imports

In [127]:
import os
import numpy as np
import pandas as pd
from pykalman import KalmanFilter

### Inputs

In [128]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [129]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

In [130]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_efsr_edge = radnet_input_df['efsr_edge'].values

### Kalman Filter

In [131]:
confidence_fusion = np.where(fusion_output_class == 1,
    fusion_output_confidence,
    1.0 - fusion_output_confidence
)

In [132]:
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=confidence_fusion[0],  # Agora usa o valor correto!
    initial_state_covariance=1.0,
    observation_covariance=1e-2,  # R
    transition_covariance=1e-2    # Q
)

In [133]:
kalman, _ = kf.filter(confidence_fusion)
kalman = kalman.flatten()

cls = (kalman >= 0.5).astype(int)
conf = np.where(cls == 1, kalman, 1.0 - kalman)

In [134]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'EFSR':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*75}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_efsr_edge[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   EFSR   Classe   Status   Confiança 
---------------------------------------------------------------------------
-75    6.1   77     0.87   0.43   0        Ruim     0.88      
-54    2.7   19     0.41   0.91   1        Bom      0.67      
-81    4.8   127    0.89   0.79   0        Ruim     0.56      
-56    2.3   14     0.41   0.95   1        Bom      0.77      
-79    4.8   18     0.85   0.92   1        Bom      0.52      
-61    1.5   6      0.27   0.91   1        Bom      0.79      
-66    4.5   79     0.87   0.39   0        Ruim     0.64      
-49    1.8   6      0.40   0.92   1        Bom      0.74      
-95    3.8   148    0.34   0.43   0        Ruim     0.65      
-55    1.6   11     0.29   0.88   1        Bom      0.72      
-49    1.1   7      0.55   0.91   1        Bom      0.86      
-72    6.3   99     0.79   0.93   1        Bom      0.56      
-95    9.8   79     0.92   0.93   0        Ruim     0.55      
-82    3.9   34     0.69   0.93   0       

### Save

In [135]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
